In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

c:\Users\ndh01\anaconda3\Lib\site-packages\torch\_subclasses\functional_tensor.py:295: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
raw_df = pd.read_csv("../data/cleaned_movies_no_outliers.csv")

In [3]:
raw_df["profit_margin"] = (raw_df["worldwide"] - raw_df["budget"]) / raw_df["budget"]
raw_df['year'] = 2024 - raw_df['years_from_release']

In [4]:
# Hợp nhất các thể loại có ít phim để giảm độ phức tạp của mô hình
merge_col = []
for col in raw_df:
    if 'genre_' in col:
        if raw_df[col].sum() < 300:
            merge_col.append(col)

raw_df['other_genre'] = raw_df[merge_col].sum(axis=1)
raw_df.drop(columns=merge_col, inplace=True)

In [5]:
data = raw_df.drop(columns=['popularity', 'title', 'vote_average', 'vote_count', 'imdb_id', 'domestic', 'international', 'worldwide', 'years_from_release'])
data

,budget,runtime,genre_Action,genre_Adventure,genre_Comedy,genre_Crime,genre_Drama,genre_Family,genre_Fantasy,genre_Horror,genre_Science_Fiction,genre_Thriller,profit_margin,year,other_genre
0,200000000,128,True,False,True,False,False,False,False,False,True,False,5.689504,2024,0
1,17500000,141,False,False,False,False,True,False,False,True,True,False,1.940764,2024,0
2,75000000,104,False,True,False,False,False,True,False,False,True,False,0.718349,2024,1
3,250000000,124,True,False,True,False,False,False,True,False,False,False,-0.660519,2024,0
4,80000000,119,False,False,False,False,False,False,False,True,True,False,3.385763,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,40000000,119,False,False,False,True,True,False,False,False,False,True,1.168809,2011,0
2301,15000000,98,False,False,False,False,False,False,True,True,False,False,-0.764379,2008,1
2302,19000000,161,True,False,False,True,True,False,False,False,False,False,-0.873809,2024,0
2303,4000000,91,False,False,True,False,False,False,False,True,False,False,2.560328,1985,0


In [6]:
data['profit_margin'].describe()

count      2305.000000
mean        113.401786
std        5121.709683
min          -0.999980
25%           0.591302
50%           2.050915
75%           4.526393
max      245865.000000
Name: profit_margin, dtype: float64

Tứ phân vị thứ 3 là khoảng 4.5 mà max lại là 245865. Cùng với độ lệch chuẩn cao bất thường cho thấy dữ liệu đang bị lệch phải nhiều. Chúng em sẽ bỏ các phần ngoại lai cụ thể là những giá trị lớn hơn Q3 + 1.5 * IQR

In [7]:
data = data[data['profit_margin'] < 10.5]

In [8]:
y = data['profit_margin'].values
X = data.drop(columns=['profit_margin']).values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print("Train profit_margin stats:")
print(pd.Series(y_train).describe())

print("\nTest profit_margin stats:")
print(pd.Series(y_test).describe())


Train profit_margin stats:
count    1676.000000
mean        2.309827
std         2.494915
min        -0.999980
25%         0.451637
50%         1.754949
75%         3.593559
max        10.391633
dtype: float64

Test profit_margin stats:
count    419.000000
mean       2.248191
std        2.432429
min       -0.999905
25%        0.441764
50%        1.659309
75%        3.411548
max       10.143583
dtype: float64


In [15]:
# Huấn luyện mô hình Random Forest
rf_model = RandomForestRegressor(n_estimators=50, max_depth=50, min_samples_split=10, min_samples_leaf=10, random_state=42)
rf_model.fit(X_train, y_train)

# Dự đoán
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Đánh giá mô hình
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train MSE: {train_mse:.4f}, Train R2: {train_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}")

Train MSE: 4.1211, Train R2: 0.3375
Test MSE: 5.4768, Test R2: 0.0721


R2 trên tập test khá thấp, cho thấy mô hình ít tổng quát được mối quan hệ giữa input và output. Điều này có thể là do dữ liệu đang hơi ít(chỉ khoảng 2000 mẫu) cho việc học.

In [12]:
def optimize_budget(budget_max, budget_min, infor_movie):
    result = 0
    max_margin = float('-inf')
    for budget in range(budget_min, budget_max + 1, int((budget_max - budget_min) / 20)):
        input = [budget] + infor_movie.copy()
        predicted_margin = rf_model.predict(np.array(input).reshape(1,-1))
        if predicted_margin > max_margin:
            result = budget
    return result

r = 5270000
infor_movie1 = [90,True,True,False,False,False,False,False,False,True,True,2024,0]
infor_movie2 = [90,1,1,0,0,0,0,0,0,1,1,2024,0]
print(optimize_budget(100000, 10000000, infor_movie1))
print(optimize_budget(100000, 10000000, infor_movie2))

595000
595000
